# TIME SERIES FORECASTING AND SENTIMENT ANALYSIS OF BIG DATA PROOCESSED WITH MYSQL VS HBASE, CASSANDRA, MONGODB

# Dataset
The dataste is a large dataset gleaned from the twitter API that is called ProjectTweets.csv.

This dataset contains 1,600,000 tweets extracted using the twitter api. 


Content
It contains the following 5 fields:
- ids: The id of the tweet (eg. 4587)
- date: the date of the tweet (eg. Sat May 16 23:58:44 UTC 2009)
- flag: The query (eg. lyx). If there is no query, then this value is NO_QUERY.
- user: the user that tweeted (eg. bobthebuilder)
- text: the text of the tweet (eg. Lyx is cool)

# STOP ANY ACTIVE SPARK SESSIONS

In [1]:
from pyspark.sql import SparkSession

# Stop all active Spark sessions
SparkSession.builder.getOrCreate().stop()

## Import all the necessary libraries required for loading the data, EDA, Data Cleaning and statistics

In [2]:
import warnings
warnings.filterwarnings("ignore")
from pyspark.ml import Pipeline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#List the contents of the root directory in HDFS
!hdfs dfs -ls /

Found 5 items
-rwxrwxrwx   1 hduser supergroup  230567856 2024-04-23 09:46 /ProjectTweets.csv
drwxr-xr-x   - hduser supergroup          0 2024-01-30 23:49 /output1
drwxrwxr-x   - hduser supergroup          0 2024-02-01 14:54 /tmp
drwxr-xr-x   - hduser supergroup          0 2024-02-01 14:48 /user
drwxr-xr-x   - hduser supergroup          0 2024-02-01 14:57 /user1


# Storing Data in a SQL vs Non SQL Database
- MySQL
- Hbase
- Cassandra
- MongoDB

# Loading data from Hadoop to mySQL and Process using Spark

This process involves a series of steps. They include:-

1. Loading the CSV file on ProjectTweets.csv data stored in hadoop.

2. Data Exploration: Perform exploratory data analysis (EDA)

This include:-

- checking missing data,
- Checking for duplicates
- Checking for oultiers
- Checking for data types.
- visualize data to see data pattern

3. Data cleaning and preparation:
- Chnage data types
- handling duplicates, missing data, outliers,
- feature selection

# Initialize Spark Session

In [6]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Load from Hadoop").getOrCreate()

# Read CSV file into DataFrame
hadoop_csv_path = "hdfs://localhost:9000/ProjectTweets.csv"

# Define column names
column_names = ["ID", "ID1", "date", "flag", "user", "text"]

df = spark.read.csv(hadoop_csv_path, header=False, inferSchema=column_names)

Py4JJavaError: An error occurred while calling o78.csv.
: java.lang.Exception: inferSchema flag can be true or false
	at org.apache.spark.sql.errors.QueryExecutionErrors$.paramIsNotBooleanValueError(QueryExecutionErrors.scala:1214)
	at org.apache.spark.sql.catalyst.csv.CSVOptions.getBool(CSVOptions.scala:96)
	at org.apache.spark.sql.catalyst.csv.CSVOptions.<init>(CSVOptions.scala:119)
	at org.apache.spark.sql.catalyst.csv.CSVOptions.<init>(CSVOptions.scala:47)
	at org.apache.spark.sql.execution.datasources.csv.CSVFileFormat.inferSchema(CSVFileFormat.scala:60)
	at org.apache.spark.sql.execution.datasources.DataSource.$anonfun$getOrInferFileFormatSchema$11(DataSource.scala:210)
	at scala.Option.orElse(Option.scala:447)
	at org.apache.spark.sql.execution.datasources.DataSource.getOrInferFileFormatSchema(DataSource.scala:207)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:411)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.

In [ ]:
df.printSchema()

In [ ]:
#Label the columns
df = df.withColumnRenamed("_c0", "ID").withColumnRenamed("_c1", "ID1").withColumnRenamed("_c2", "date").withColumnRenamed("_c3", "flag").withColumnRenamed("_c4", "user").withColumnRenamed("_c5", "text")

In [ ]:
 df.show(5)

In [ ]:
#View the features present in the DataFrame
df.printSchema()

In [ ]:
type(df)

In [ ]:
# Define JDBC connection properties
mysql_properties = {
    "driver": "com.mysql.jdbc.Driver",
    "url": "jdbc:mysql://localhost:3306/ProjectTweets",
    "user": "hduser",
    "password": "hadoop"
}

# Define the MySQL table name
mysql_table_name = "ProjectTweets"

# Read data from MySQL into DataFrame
df_mysql = spark.read \
    .jdbc(url=mysql_properties["url"],
          table=mysql_table_name,
          properties=mysql_properties)


In [ ]:
#Create a temporary view using the DataFrame
df.createOrReplaceTempView("tweets")

In [ ]:
spark.sql("select * from df").collect()

In [ ]:
# Transform and clean the data if necessary
# For example:
# df = df.withColumnRenamed("old_column_name", "new_column_name")

# Write DataFrame to MySQL table
mysql_properties = {
    "driver": "com.mysql.jdbc.Driver",
    "url": "jdbc:mysql://your_mysql_host:your_mysql_port/your_database_name",
    "user": "your_username",
    "password": "your_password"
}
mysql_table_name = "your_table_name"

df.write.jdbc(url=mysql_properties["url"],
              table=mysql_table_name,
              mode="overwrite",  # Specify the write mode
              properties=mysql_properties)

# Stop SparkSession
spark.stop()


# MySQL via Pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [ ]:
# Initialize SparkSession
sc = SparkSession.builder.appName("Load CSV to MySQL").getOrCreate()


In [ ]:
# Initialize SparkSession
sc = SparkSession.builder.appName("Load CSV to MySQL").getOrCreate()

In [ ]:
# Read CSV file into DataFrame
csv_path = "file:///home/hduser/ProjectTweets.csv"
df = sc.read.csv(csv_path, header=False, inferSchema=True)

In [ ]:
df.printSchema()

In [ ]:
df = df.withColumnRenamed("_c0", "ID").withColumnRenamed("_c1", "ID2").withColumnRenamed("_c2", "date").withColumnRenamed("_c3", "flag").withColumnRenamed("_c4", "user").withColumnRenamed("_c5", "text")

In [ ]:
df.printSchema()

In [ ]:
# View the Spark DataFrame
df.show(5)

In [ ]:
display(df)

# Storing and Processing Data in MySQL using pyspark

In [ ]:
# Define JDBC connection properties
mysql_properties = {
    "driver": "com.mysql.jdbc.Driver",
    "url": "jdbc:mysql://localhost:3306/ProjectTweets",
    "user": "hduser",
    "password": "hadoop"
}

# Define MySQL table name
mysql_table_name = "ProjectTweets"

# Read CSV file into DataFrame
csv_path = "file:///home/hduser/ProjectTweets.csv"
df = sc.read.csv(csv_path, header=False, inferSchema=True)

# Write DataFrame to MySQL table
df.write.jdbc(url=mysql_properties["url"],
              table=mysql_table_name,
              mode="overwrite",  # Specify the write mode
              properties=mysql_properties)

# Stop SparkSession
spark.stop()


# HBASE DATABASE

# Import the CSV to HBase